In [1]:
import re
import math
import sys
import random

import numpy
import pandas

import ipywidgets as widgets
from IPython.display import display, clear_output

import gensim
from gensim.corpora import Dictionary
from gensim.models import TfidfModel

import pythainlp

from sklearn.metrics.pairwise import cosine_similarity

from data_tokenizer import load_corpus

### Load Data

In [2]:
file_name = 'ผู้บริโภค - TescoLotus.txt'

corpus, labels = load_corpus('../data/facebook/' + file_name)

len_corpus = len(corpus)
print('Total documents', len_corpus)

clusters = list(set(labels))
print(len(clusters), 'clusters')

f = open('../data/facebook/tokenized/tokenized_' + file_name)
tokenized_corpus = eval(f.read())
f.close()

Total documents 269
1 clusters


### Preprocess Corpus

#### Remove Words

In [3]:
dictionary = Dictionary(tokenized_corpus)
print('origin:', len(dictionary), 'words')

dictionary.filter_extremes(no_below=2, no_above=0.7, keep_n=len(dictionary))
print('filter frequent words:', len(dictionary), 'words')

letter_words = [id for id in range(len(dictionary)) if len(dictionary[id]) <= 1] 
dictionary.filter_tokens(bad_ids=letter_words)
print('filter letter words:', len(dictionary), 'words')

stopwords = pythainlp.corpus.stopwords.words('thai')
stopwords.append('นี้')
dictionary.add_documents([stopwords])
stopwords = [dictionary.token2id[word] for word in stopwords]
dictionary.filter_tokens(bad_ids=stopwords)
print('filter stop words:', len(dictionary), 'words')

origin: 1313 words
filter frequent words: 540 words
filter letter words: 539 words
filter stop words: 352 words


In [4]:
# bow_corpus = [dictionary.doc2bow(doc) for doc in tokenized_corpus]
idx_corpus = [dictionary.doc2idx(doc) for doc in tokenized_corpus]

temp_corpus = []
for doc in idx_corpus:
    temp_corpus.append([dictionary[id] for id in doc if id >= 0])
idx_corpus = temp_corpus

#### Dimension Reduction

In [5]:
average_doc_size = 0
for doc in idx_corpus:
    average_doc_size += len(doc)
average_doc_size /= len(idx_corpus)
average_doc_size = math.ceil(average_doc_size)
average_doc_size

df = dictionary.dfs
filtered_corpus = []
for doc in idx_corpus:
    new_doc = [(word, df[dictionary.token2id[word]]) for word in doc]
    new_doc.sort(reverse=True, key=lambda x: x[1])
    new_doc = new_doc[:average_doc_size]
    filtered_corpus.append([word for word, df in new_doc])

### SDC

In [54]:
def get_bow(corpus):
    new_dict = Dictionary(corpus)

    # new_dict.filter_extremes(no_below=2, no_above=1, keep_n=len(new_dict))
    # print(len(new_dict))

    unique_words = [new_dict[id] for id in range(len(new_dict))]
    array = numpy.zeros((len_corpus, len(unique_words)), dtype=float)
    
    for i, doc in enumerate(corpus):
        for word in doc:
            array[i, new_dict.token2id[word]] += 1

        ## normalization
        if len(doc) != 0:
            array[i] = numpy.divide(array[i], len(idx_corpus[i]))

    return pandas.DataFrame(array, columns=unique_words, dtype=float)

In [7]:
def sdc(bow_corpus, min_samples, eps):
    delta_eps = eps / 10
    labels = [-1 for i in range(len(bow_corpus))]
    sims = cosine_similarity(bow_corpus)
    
    points = [i for i in range(len(bow_corpus))]
    cluster_num = 0
    while len(points) > 0:
        seed = random.choice(points)
        eps_neighbors = [i for i, sim in enumerate(sims[seed]) if sim >= eps and labels[i] == -1]
        if len(eps_neighbors) >= min_samples:
            cluster_num += 1
            for p in eps_neighbors:
                labels[p] = cluster_num
            points = [i for i in points if i not in eps_neighbors]
        else:
            labels[seed] = 0
            points.remove(seed)

    while cluster_num != 0:
        cluster = [numpy.array(bow_corpus.iloc[i]) for i, label in enumerate(labels) if label == cluster_num]
        eps_temp = eps
        
        while True:
            centroid = numpy.mean(cluster, axis=0).reshape(1, -1)
            eps_temp -= delta_eps
            
            count = 0
            for i, label in enumerate(labels):
                point = numpy.array(bow_corpus.iloc[i]).reshape(1, -1)
                if label == 0 and cosine_similarity(centroid, point) >= eps_temp:
                    cluster.append(point[0])
                    labels[i] = cluster_num
                    count += 1
            if count == 0:
                break
        
        cluster_num -= 1
    
    return labels

In [8]:
def upgrade_sdc(bow_corpus, min_samples, eps):
    labels = [-1 for i in range(len(bow_corpus))]
    initials = []
    delta_eps = eps / 10
    sims = cosine_similarity(bow_corpus)
    
    points = [i for i in range(len(bow_corpus))]
    clusters = []
    cluster_num = 0
    clusters.append([])
    while len(points) > 0:
        seed = random.choice(points)
        eps_neighbors = [i for i, sim in enumerate(sims[seed]) if sim >= eps and labels[i] == -1]
        if len(eps_neighbors) >= min_samples:
            print(seed)
            cluster_num += 1
            clusters.append([])
            for p in eps_neighbors:
                initials.append(p)
                labels[p] = cluster_num
                clusters[cluster_num].append(numpy.array(bow_corpus.iloc[p]))
            points = [i for i in points if i not in eps_neighbors]
        else:
            labels[seed] = 0
            clusters[0].append((seed, numpy.array(bow_corpus.iloc[seed])))
            points.remove(seed)
            
    expandable = numpy.zeros(cluster_num + 1)    
    while numpy.sum(expandable) != -cluster_num - 1:
        eps -= delta_eps
        count = numpy.zeros(cluster_num + 1)
        for point in clusters[0]:
            if labels[point[0]] != 0:
                continue

            num = point[0]
            p = point[1].reshape(1, -1)
            sim = 0
            for c, cluster in enumerate(clusters[1:]):
                if expandable[c + 1] == -1:
                    continue
                centroid = numpy.mean(cluster, axis=0).reshape(1, -1)
                if cosine_similarity(centroid, p) >= sim:
                    sim = cosine_similarity(centroid, p)
                    if sim >= eps:
                        labels[num] = c + 1

            if labels[num] != 0:
                count[labels[num]] += 1
                clusters[labels[num]].append(point[1])

        for i, num in enumerate(count):
            if num == 0:
                expandable[i] = -1
        
    return labels, initials

In [9]:
def predict_cluster(bow_corpus, min_samples, eps):
    predicted_labels, initials = upgrade_sdc(bow_corpus, min_samples, eps)

    result = pandas.DataFrame()
    result['comment'] = corpus
    result['tokenized_comment'] = filtered_corpus
    result['label'] = labels
    result['predicted_label'] = predicted_labels
    
    return result, initials

In [51]:
# bow_corpus = get_bow(idx_corpus)
bow_corpus = get_bow(filtered_corpus)
result, initials = predict_cluster(bow_corpus, 7, 0.3)
bow_corpus.head()

211
52
30
227
48
9
187
27
13
83
213


,ทำ,พนักงาน,ข้าว,ดี,น้ำ,อี,เครื่อง,เลิก,กก,งง,...,พลังงาน,คอย,แอร์ไม่,สลิป,คู,ปอง,นวมินทร์,สิบ,ห้า,แจก
0,0.333333,0.666667,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.000000,0.000000,0.166667,0.166667,0.166667,0.166667,0.166667,0.166667,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.000000,0.000000,0.000000,0.000000,0.111111,0.000000,0.000000,0.000000,0.111111,0.111111,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.111111,0.111111,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


### Result

In [52]:
label_count = numpy.unique(result['predicted_label'], return_counts=True) 
num_cluster = label_count[0][-1] + 1
print(label_count)

clusters = list(set(labels))
for cluster in clusters:
    print('\t' + cluster, end='')
print('\tpercent')

for label in range(len(clusters)):
    print(str(label) + '  |', end='')
    
    num_max = 0
    for cluster in clusters:
        loc = result[(result['label'] == cluster) & (result['predicted_label'] == label)]
        if len(loc) > num_max:
            num_max = len(loc)
        print('\t' + str(len(loc)), end='')
    
    print('\t' + str(num_max / label_count[1][label]))

(array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11]), array([49, 20, 41, 11, 14, 28, 23, 29, 17, 18, 12,  7]))
		percent
0  |	49	1.0


In [53]:
clusters = [[] for i in range(num_cluster)]
corpus_centroid = []
for i, label in result['predicted_label'].iteritems():
    clusters[label].append(numpy.array(bow_corpus.iloc[i]))
    corpus_centroid.append(numpy.array(bow_corpus.iloc[i]))
corpus_centroid = numpy.mean(corpus_centroid, axis=0).reshape(1, -1)   

print('\tIntra cluster sim\tInter cluster sim\tIntra / Inter')
summation = 0
for i in range(num_cluster):
    size = len(clusters[i])
    if size != 0:
        print(i, end='\t')
        centroid = numpy.mean(clusters[i], axis=0).reshape(1, -1)
        similarities = cosine_similarity(centroid, clusters[i])
        pairwises = cosine_similarity(clusters[i])
        intra = numpy.sum(similarities) / size
        inter = cosine_similarity(centroid, corpus_centroid)[0][0]
        print(intra, end='\t')
        print(inter, end='\t')
        print(intra / inter)
        summation += numpy.sum(similarities)
    print()
print('Summation', summation)

	Intra cluster sim	Inter cluster sim	Intra / Inter
0	0.18976292231398123	0.3077350697865628	0.6166437983347038

1	0.5719088054130818	0.4661529459794041	1.2268694434859375

2	0.5998779276320713	0.7565704836939566	0.7928910003244725

3	0.6091824129700055	0.42081422393142054	1.4476279040161035

4	0.599985469966392	0.2571322815767566	2.3333727927401067

5	0.47285481351699	0.6572732240362573	0.719418951244096

6	0.536163626756667	0.6819793568043518	0.7861874724024573

7	0.6420237248737604	0.4398735723635048	1.4595642139264549

8	0.5558531295636603	0.33108369796106113	1.6788900600869643

9	0.45710217840787926	0.4585892053968852	0.996757387719759

10	0.5577752498950477	0.66456528593189	0.8393084347054097

11	0.5767160558023446	0.4410188201198887	1.3076903512769984

Summation 133.03040147526033


In [13]:
comment_widget = widgets.ToggleButtons(
    options=[num for num in range(num_cluster)],
    disabled=False,
    button_style='',
)

def on_comment_widget_click(change):
    clear_output()
    display(comment_widget)
    for index, value in result[result['predicted_label'] == change['new']]['comment'].iteritems():
        if index in initials:
            print("*", end="")
        print(index, value)

comment_widget.observe(on_comment_widget_click, names='index')
on_comment_widget_click({'new' : 0})

ToggleButtons(options=(0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11), value=0)

1 เลิกเปิดเพลง ข้าวแสนดี กับอีเครื่องกรองน้ำเพียว ได้แล้ว
5 อย่าบังคับน้องเข้าประชุมเชียร์ อย่าลงโทษโดยเหตุผลงี่เง่าๆ เกิดก่อนไม่กี่ปีเอง
20 เช็คเอาท์  หน้าจะงอไปไหนสวัสดีค่ะมีอะไรให้ช่วยมั้ยคะ
26 โลตลาด พนงปากตลาด นินทาลคเผาขน ตะโกนโหวกเหวกข้ามหัวลค วันหวยออกสนั่นเป็นพิเศษ เห็นพฤติกรรมแล้วแย่ ปรับปรุงภาพลักษณ์เถอะ
32 สาขายโสธรเปิดใหม่ พนงแอบโกง   ลูกค้ากุเอง จ่ายหลายใบแล้วทำเนียนว่ามีใบนึงชำรุดใช้ไม่ได้แล้วพอขอคืนก็สลับใบที่รูดใช้แล้วมาแทน แจ้งผู้จัดการขอดูกล้องวงจรปิดไม่ยอมให้ดูเลยต้องขู่ว่าจะแจ้งความถึงยอมคืนให้ แต่ไม่มีคำขอโทษใดๆ จาก พนงติด
57 ของป้ายเหลืองพวกผัก เน่าจนไม่รู้จะเน่ายังไง คือทิ้งๆไปก็ได้มั่ง
59 กลิ่นของห้างค่ะ ที่สาสขาบางซื่อ คือ มันทั้งอาหาร ความชื้น ฝุ่น  แอร์สกปรกๆๆรวมกัน
79 เลิกก๊อปปี้สินค้าแบรนด์ แล้วทำคุณภาพห่วยๆ แม่งเล่นเลย์ซะเหมือนเลย สาดดด
91 โลตัส รัตนาธิเบศร์ แอร์เหม็นมาก เหม็นสาบหนูสุดๆ ตรงโซนอาหารแห้ง บะหมี่กึ่งสำเร็จรูป เดินผ่านก็รู้เลยมีหนู ตรงโซนข้าวสารอีก
102 ไม่รุ้ทำไมเป้นแบบนี้แย่เรื่องมารยาทจริงๆไม่ได้เท่าขี้เล็บ7เลย
104 ของเซลคือเซลเเต่หมดอายุมาห

In [14]:
token_widget = widgets.ToggleButtons(
    options=[num for num in range(num_cluster)],
    disabled=False,
    button_style='',
)

def on_token_widget_click(change):
    clear_output()
    display(token_widget)
    for index, value in result[result['predicted_label'] == change['new']]['tokenized_comment'].iteritems():
        print(index, value)

token_widget.observe(on_token_widget_click, names='index')
on_token_widget_click({'new' : 0})

ToggleButtons(options=(0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11), value=0)

1 ['ดี', 'เลิก', 'น้ำ', 'เครื่อง', 'ข้าว', 'อี']
5 ['อย่า', 'อย่า', 'กี่', 'ปี', 'บังคับ', 'น้อง', 'เชียร์', 'งี่', 'เง่า']
20 ['หน้า', 'เช็ค', 'งอ', 'สวัสดี']
26 ['ปรับปรุง', 'พนง', 'แย่', 'ตะโกน', 'ข้าม', 'หวย', 'พิเศษ', 'พฤติกรรม']
32 ['ลูกค้า', 'จ่าย', 'พนง', 'พนง', 'ใบ', 'ใบ', 'ใบ', 'ติด', 'นึง']
57 ['รู้', 'ผัก', 'เน่า', 'เน่า', 'ทิ้ง', 'ป้ายเหลือง']
59 ['ห้าง', 'อาหาร', 'กลิ่น', 'ฝุ่น']
79 ['ทำ', 'เหมือน', 'สินค้า', 'เลิก', 'แม่ง', 'ห่วย', 'คุณภาพ']
91 ['เดิน', 'รู้', 'อาหาร', 'เหม็น', 'ข้าว', 'หนู', 'หนู', 'โซน', 'แห้ง']
102 ['เรื่อง', 'มารยาท', 'แย่', 'ขี้', 'เล็บ']
104 ['กิน', 'เดือน', 'หรอ', 'เเต่', 'อายุ', 'เเล้ว']
112 ['หมู', 'หมู', 'หมู', 'ไก่', 'ไก่', 'ไก่', 'ใหน', 'ตั้งใจ', 'ชื้อ']
123 ['คน', 'รู้', 'โดน', 'มือ', 'ก้อ', 'บังคับ', 'ไหว้', 'ไหว้', 'ไหว้']
131 ['ชอบ', 'เอ็กเพรส', 'แลก', 'ไหม', 'แสตมป์', 'โทร']
133 ['ทำ', 'อาหาร', 'ดู', 'สด', 'สกปรก', 'พื้น', 'พื้นที่', 'โซน', 'ลาก']
140 ['ติด', 'บัตร', 'เครื่อง', 'หาย']
156 ['หน้า', 'น้ำ', 'เย็น', 'กี่', 'เหม็น', 'ชั้น', '

In [15]:
seed = 244
compare = 89

a = numpy.array(bow_corpus.iloc[seed]).reshape(1, -1)
b = numpy.array(bow_corpus.iloc[compare]).reshape(1, -1)
print(cosine_similarity(a,b))
print(filtered_corpus[seed])
print(filtered_corpus[compare])

[[0.15811388]]
['สาขา', 'ลูกค้า', 'โลตัส', 'ถุง', 'นวมินทร์']
['ถุง', 'ซื้อ', 'เดิน', 'ซ้อน', 'ผ้า', 'นุ่ม', 'เชียร์', 'ถุงขาด']
